## Abstract
In this project we implement two methods for image blending : Poisson method and multiresolution
blending with Laplacian stack. Main functions for these methods are implemented in
`poisson_blend.py` and `multiresolution_blend.py`, and the code for testing methods on
sample images is in `main.py`

### Poisson Blending
Target and source images both have n pixels, and we also have a mask. We create a vector matrix b
with n elements, each element corresponding to one pixel in target.
If a pixel is in the mask, its corresponding element in b will be the Laplacian of
source in that pixel. Otherwise, it will be the value of that pixel in target.
After creating b, we also create an $n \times n$ sparse matrix $A$,
which is the coefficient matrix of the equations used for computing final image.
Since $A$ is sparse, we use sparse matrix data structure implemented in `scipy.sparse` for
working with it.
Finally, we solve the least square error problem $Af=b$. Result $n \times 1$ vector
$f$ contains the value of each pixel in the blended image.


In [ ]:
import numpy as np
import cv2
import scipy.sparse.linalg as sparse_la
from scipy import sparse


First function simply computes the Laplacian of an image:

In [ ]:
def get_laplacian(img):
    kernel = np.asarray([[0, -1, 0], [-1, 4, -1], [0, -1, 0]], dtype='float32')
    result = cv2.filter2D(img, ddepth=-1, kernel=kernel)
    return result


Function `generate_matrix_b` creates matrix $b$, just in the way explained in the begining:



In [ ]:
def generate_matrix_b(source, target, mask):
    source_laplacian_flatten = get_laplacian(source).flatten('C')
    target_flatten = target.flatten('C')
    mask_flatten = mask.flatten('C')
    b = (mask_flatten) * source_laplacian_flatten + (1 - mask_flatten) * target_flatten
    return b


`generate_matrix_A` creates coefficient matrix $A$:

In [ ]:
def generate_matrix_A(mask):
    data, cols, rows = [], [], []
    h, w = mask.shape[0], mask.shape[1]
    mask_flatten = mask.flatten('C')
    zeros = np.where(mask_flatten == 0)
    ones = np.where(mask_flatten == 1)
    # adding ones to data
    n = zeros[0].size
    data.extend(np.ones(n, dtype='float32').tolist())
    rows.extend(zeros[0].tolist())
    cols.extend(zeros[0].tolist())

    # adding 4s to data
    m = ones[0].size
    data.extend((np.ones(m, dtype='float32') * (4)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend(ones[0].tolist())

    # adding -1s
    data.extend((np.ones(m, dtype='float32') * (-1)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend((ones[0] - 1).tolist())

    data.extend((np.ones(m, dtype='float32') * (-1)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend((ones[0] + 1).tolist())

    data.extend((np.ones(m, dtype='float32') * (-1)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend((ones[0] - w).tolist())

    data.extend((np.ones(m, dtype='float32') * (-1)).tolist())
    rows.extend(ones[0].tolist())
    cols.extend((ones[0] + w).tolist())
    return data, cols, rows

Next function solves the LSE problem $Af=b$ and returns $f$

In [ ]:
def solve_sparse_linear_equation(data, cols, rows, b, h, w):
    sparse_matrix = sparse.csc_matrix((data, (rows, cols)), shape=(h * w, h * w), dtype='float32')
    f = sparse_la.spsolve(sparse_matrix, b)
    f = np.reshape(f, (h, w)).astype('float32')
    return f

Final function `blend_images` takes source,target and mask as input and applies blending, using above functions :

In [ ]:
def blend_images(source, target, mask):
    h, w = source.shape[0], source.shape[1]
    source_b, source_g, source_r = cv2.split(source)
    target_b, target_g, target_r = cv2.split(target)
    data, cols, rows = generate_matrix_A(mask)
    b_b = generate_matrix_b(source_b, target_b, mask)
    b_g = generate_matrix_b(source_g, target_g, mask)
    b_r = generate_matrix_b(source_r, target_r, mask)
    blended_b = solve_sparse_linear_equation(data, cols, rows, b_b, h, w)
    blended_g = solve_sparse_linear_equation(data, cols, rows, b_g, h, w)
    blended_r = solve_sparse_linear_equation(data, cols, rows, b_r, h, w)
    result = cv2.merge((blended_b, blended_g, blended_r))
    return result

### Multiresolution Blending and Feathering
We blend two given images using Laplacian stack and feathering.
Algorithm is simple: first we create Gaussian stack for each image, then create
Laplacian stack using Gaussian stack. We also create a stack of masks for blending.
Then we blend the images in each level of the stack using their corresponding mask to get blended stack. Finally,
We sum up all images in blended stack to get the final result.


First function simply uses GaussianBlur method in opencv to blur a given image :

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.signal as signal


def blur_img(img, kernel_size, sigma):
    result = cv2.GaussianBlur(img, (kernel_size, kernel_size), sigma)
    return result

Next two function generate Gaussian and Laplacian stack for a given image :

In [ ]:
def generate_gaussian_stack(img, kernel_size, sigma, stack_size):
    result = []
    result.append(img)
    temp_img = img.copy()
    for i in range(1, stack_size):
        temp_img = blur_img(temp_img, kernel_size, sigma)
        result.append(temp_img)
    return result

def generate_laplacian_stack(img, kernel_size, sigma, stack_size):
    gaussian_pyramid = generate_gaussian_stack(img, kernel_size, sigma, stack_size)
    result = []
    for i in range(0, stack_size - 1):
        temp = gaussian_pyramid[i].copy() - gaussian_pyramid[i + 1].copy()
        result.append(temp)
    result.append(gaussian_pyramid[-1].copy())
    return result



Function `generate_mask` creates a stack of masks. Each element of the stack is the result of
blurring previous element, so in higher levels, we will have more intensive feathering

In [ ]:
def generate_masks(mask, kernel_size, sigma, size):
    result = []
    new_mask = blur_img(mask, kernel_size, sigma)
    result.append(new_mask)
    for i in range(size):
        new_mask = blur_img(new_mask, kernel_size, sigma)
        # plt.imshow(new_mask)
        # plt.show()
        result.append(new_mask)

    return result



Next function creates a stack of blended images, in the way described
in the begining :

In [ ]:
def generate_blended_stack(img1, img2, masks, kernel_size, sigma, stack_size):
    result = []
    laplacian_stack1 = generate_laplacian_stack(img1, kernel_size, sigma, stack_size)
    laplacian_stack2 = generate_laplacian_stack(img2, kernel_size, sigma, stack_size)
    for i in range(stack_size):
        blended_img = laplacian_stack1[i] * masks[i] + laplacian_stack2[i] * (1 - masks[i])
        result.append(blended_img)
    return result

Final function `blend_images`, takes two images, a stack of masks, a kernel size and sigma for blurring and stack size
as input, and outputs the resulting blended image:

In [ ]:
def blend_images(img1, img2, masks, kernel_size, sigma, stack_size):
    blended_stack = generate_blended_stack(img1, img2, masks, kernel_size, sigma, stack_size)
    result = blended_stack[0].copy()
    for i in range(1, stack_size):
        result = result + blended_stack[i]
    return result


Function `convert_from_float32_to_uint8` is used to scale an image
from interval [0,1] to interval [0,255], with integer valued pixels :

In [ ]:
def convert_from_float32_to_uint8(img):
    max_index, min_index = np.max(img), np.min(img)
    a = 255 / (max_index - min_index)
    b = 255 - a * max_index
    result = (a * img + b).astype(np.uint8)
    return result

### main.py
In the main file, after reading images, we scale them to the range [0,1], apply above functions on them and rescaling the result
back to range [0,255]. This process is done both for sample images used in Poisson blending,
and multiresolution blending.

Test code for Poisson blending:


In [ ]:
import numpy as np
import cv2
import poisson_blend
import multiresolution_blend

''' applying poisson blending '''
# reading images and scaling them to [0,1]
img1 = cv2.imread("inputs/cards.jpg")
img2 = cv2.imread("inputs/desk.jpg")

img1 = (img1 / 255).astype('float32')
img2 = (img2 / 255).astype('float32')

source = img1[195:485, 355:715, :].astype('float32')
target = img2[195:485, 355:715, :].astype('float32')

mask = np.zeros((290, 360), dtype='float32')
mask[5:-5, 5:-5] = 1

max_matrix, min_matrix = np.ones(source.shape, dtype='float32') * 255, np.zeros(source.shape, dtype='float32')
result = poisson_blend.blend_images(source, target, mask) * 255
result = np.minimum(result, max_matrix)
result = np.maximum(result, min_matrix)
result = result.astype('uint8')
final_result = (img2.copy() * 255).astype('uint8')
final_result[195:485, 355:715, :] = result
cv2.imwrite("outputs/cards-on-desk.jpg", final_result)

Test code for multiresolution blending:

In [ ]:
''' applying multiresolution blending '''
img1 = cv2.imread("inputs/tree_spring.jpg")
img2 = cv2.imread("inputs/tree_fall.jpg")

img_float1 = (img1 / 255).astype('float32')
img_float2 = (img2 / 255).astype('float32')

kernel_size, sigma, n = 151, 30, 10
img_list = multiresolution_blend.generate_gaussian_stack(img1, kernel_size, sigma, n)
lap_list = multiresolution_blend.generate_laplacian_stack(img1, kernel_size, sigma, n)

''' creating masks '''
h, w = img1.shape[0], img1.shape[1]
mask = np.zeros(img1.shape, dtype='float32')
mask[:, 0:w//2] = 1
masks = multiresolution_blend.generate_masks(mask, 151, 30, n)

result = multiresolution_blend.blend_images(img_float1, img_float2, masks, kernel_size, sigma, n)
result = multiresolution_blend.convert_from_float32_to_uint8(result)
cv2.imwrite("outputs/tree-spring-fall.jpg", result)

You can see the results of above codes in outputs folder.